In [ ]:
# import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorflow_examples.models.pix2pix import pix2pix
# !pip install git+https://github.com/tensorflow/examples.git


from IPython.display import clear_output
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
import random
import os
import sys

In [ ]:
sys.path.append("../models/")
from models.models import generate_model_one

In [ ]:
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 1000
PLOT_FLAG = True

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

NUMBER_OF_DATA_POINTS = 1000

In [ ]:
# to load data; first to enable gdrive
# https://neptune.ai/blog/google-colab-dealing-with-files
# https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/

from google.colab import drive
# drive.mount('/content/gdrive')

drive.flush_and_unmount()
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# !ls gdrive/MyDrive/rooftop 

In [ ]:
np.random.seed = RANDOM_SEED

len_data = NUMBER_OF_DATA_POINTS
len_train_data = int(0.8 * len_data)

PATH = "gdrive/MyDrive/rooftop/data_{}".format(len_data )
base_directory = PATH
images_folder = os.path.join(base_directory, 'train_imgs_{}'.format(len_data))
masks_folder = os.path.join(base_directory, 'mask_imgs_{}'.format(len_data))

In [ ]:
X = np.zeros((len_data, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len_data, IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print(X.shape, y.shape)

In [ ]:
def transform_img(img):
    return resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)

In [ ]:
FOLDER_TRAIN = "train_imgs_{}".format(len_data)
FOLDER_MASKS = "mask_imgs_{}".format(len_data)

def fn_create_data_sets(DEBUG: bool = False):
  data_images = []
  data_segmentation_masks = []
  data_file_names = []
  
  for cnt, file_name in enumerate(os.listdir(os.path.join(PATH, FOLDER_TRAIN))):
      file_path = os.path.join(PATH, FOLDER_TRAIN, file_name)
      mask_file_name = "{}_{}".format("mask", file_name)
      mask_file_path = os.path.join(PATH, FOLDER_MASKS, mask_file_name)

      if os.path.isfile(file_path) and os.path.isfile(mask_file_path):
          image = Image.open(file_path),
          segmentation_mask = Image.open(mask_file_path),

          data_images.append(image)
          data_segmentation_masks.append(segmentation_mask)
          data_file_names.append(file_name)
      
      if cnt % 100 == 0: print("cnt:", cnt)
      if DEBUG and cnt > 1000: break
      
  return (data_images, data_segmentation_masks, data_file_names)

In [ ]:
# os.listdir(os.path.join(PATH, FOLDER_TRAIN))

In [ ]:
data_images, data_segmentation_masks, data_file_names = fn_create_data_sets()

In [ ]:
PLOT_FLAG

In [ ]:
data_images[0][0]

In [ ]:
data_segmentation_masks[0][0]

In [ ]:
print(
    np.array(data_images[0][0]).shape, 
    np.array(data_segmentation_masks[0][0]).shape,
)

In [ ]:
def resize_img_tf(input_img):
    return tf.image.resize((tf.cast(input_img, tf.float32) / 255.0), (128, 128))

def transformation_of_mask(input_mask):
  # convert to gray scale
  gray_img = input_mask.convert('L')
  transformed_gray_img = np.array((np.array(gray_img) > 255 // 2) * 255)
  gray_to_tensor = tf.reshape(
    tf.convert_to_tensor(transformed_gray_img),
    (300, 300, 1),
  )
  result = tf.image.resize(
      gray_to_tensor,
      (128, 128),
      method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    )
  
  # important: do we return: (result) or (1 - result)
  # return 1 - result
  
  return result

In [ ]:
plt.imshow(resize_img_tf(data_images[0][0]))

In [ ]:
plt.imshow(
  transformation_of_mask(data_segmentation_masks[0][0]),
  cmap=plt.cm.binary,
)

In [ ]:
input_mask = data_segmentation_masks[0][0]
# convert to gray scale
gray_img = input_mask.convert('L')
transformed_gray_img = np.array((np.array(gray_img) > 255 // 2) * 255)
gray_to_tensor = tf.reshape(
  tf.convert_to_tensor(transformed_gray_img),
  (300, 300, 1),
)
result = tf.image.resize(
    gray_to_tensor,
    (128, 128),
    method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
)


In [ ]:
input_mask

In [ ]:
gray_img

In [ ]:
gray_to_tensor

In [ ]:
plt.imshow(np.array(1 - gray_to_tensor))

In [ ]:
plt.imshow(1 - result, cmap=plt.cm.binary)

In [ ]:
# from PIL import Image
# data_images[0][0].resize((IMG_HEIGHT, IMG_WIDTH), Image.Resampling.LANCZOS)

In [ ]:
X_data = [resize_img_tf(img[0]).numpy() for img in data_images]
y_data = [transformation_of_mask(img[0]).numpy() for img in data_segmentation_masks]

In [ ]:
X = np.zeros((len_data, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y = np.zeros((len_data, IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)


In [ ]:
for i in range(len_data):
  X[i] = X_data[i]
  y[i] = y_data[i]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print(X_train.shape, y_train.shape, y_train.shape, y_test.shape)

In [ ]:
# X_data[0]
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)


In [ ]:
from models.models import generate_model_one

(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = generate_model_one(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.summary()

In [ ]:
callbacks = [
        # tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    batch_size=16, 
    epochs=2, 
    callbacks=callbacks,
)

In [ ]:
def get_utc_timestamp_formatted(time_format: str = "%Y%m%d_%H%M%S") -> str:
    from datetime import datetime
    return datetime.utcnow().strftime(time_format)

utc_timestamp = get_utc_timestamp_formatted()

In [ ]:
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

plt.figure()
plt.plot( loss, 'r', label='Training loss')
plt.plot( val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.grid(True)
plt.savefig("results_train_val_{}".format(NUMBER_OF_DATA_POINTS, utc_timestamp))
if PLOT_FLAG:
  plt.show()

In [ ]:
accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']

plt.figure()
plt.plot( accuracy, 'r', label='Training accuracy')
plt.plot( val_accuracy, 'bo', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.grid(True)
plt.savefig("results_train_val_{}".format(NUMBER_OF_DATA_POINTS, utc_timestamp))
if PLOT_FLAG:
  plt.show()

In [ ]:
OUTPUT_CLASSES = 2

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
callbacks = [
        # tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    batch_size=16, 
    epochs=20, 
    callbacks=callbacks,
)

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    # display([sample_image, sample_mask,
    #          create_mask(model.predict(sample_image[tf.newaxis, ...]))])
    plt.imshow(sample_image)
    plt.imshow(sample_mask)
    plt.imshow(
        create_mask(
            model.predict(sample_image[tf.newaxis, ...])
            )
        )

    


In [ ]:
sample_image = data_images[100][0]
sample_mask = data_segmentation_masks[100][0]
# show_predictions()
# create_mask(model.predict(sample_image[tf.newaxis, ...])
        


In [ ]:
plt.imshow(sample_image)

In [ ]:
plt.imshow(sample_mask)

In [ ]:
# plt.imshow(create_mask(model.predict(sample_image[tf.newaxis, ...])))